In [1]:
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pandas as pd

# Generating a Convenient Science-Ready Dataframe


In this notebook, I compile all the data being collected from various sources into one dataframe with which we can conduct our science. The data involved is as follows:

    1. SDSS x ALMA matched target list from xmatch_Oct2024.ipynb
    2. Gass masses from salvage-post-processing.ipynb
    3. Visual classifications of the images and spectra

## 1. Load in the SALVAGE target list

In [7]:
fpath = '/arc/projects/salvage/ALMA_reduction/samples/'
file  = 'salvage-Sep20-sample_match-lt-4rp_mrs-gt-1p5rp_res-lt-3p05_qa2-pass_no-inf-dups.txt'

objID_sample, year_sample, muid_sample, auid_sample, proj_sample, name_sample = np.loadtxt(fpath+file, unpack = True, dtype = str, usecols = [0,9,10,11,12,13])
z_sample, mass_sample, rpetro_sample, ra_sample, dec_sample, res_sample, mrs_sample, sens_sample  = np.loadtxt(fpath+file, unpack = True, dtype = float, usecols = [1,2,3,4,5,6,7,8])

FileNotFoundError: /arc/projects/salvage/ALMA_reduction/samples/salvage-Sep20-sample_match-lt-4rp_mrs-gt-1p5rp_res-lt-3p05_qa2-pass_no-inf-dups.txt not found.

In [4]:
d = {'objID': objID_sample, 'source_name': name_sample, 'project_code': proj_sample, 'year': year_sample, 'MUID': muid_sample, 'AUID': auid_sample, \
    'ra': ra_sample, 'dec': dec_sample, 'z': z_sample, 'mstar_total': mass_sample, 'rpetro': rpetro_sample, 'MRS': mrs_sample, 'resolution': res_sample}
df = pd.DataFrame(data=d)
df

NameError: name 'WISE' is not defined

## 2. Load in Molecular Gas Masses computed from ALMA Cubes

In [11]:
fpath = '/arc/projects/salvage/ALMA_reduction/'
file = 'salvage-AGN-Oct10-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups_ellipap.out'
file = 'salvage-AGN-Oct14-sample_match-lt-4rp_mrs-gt-2rp_qa2-pass_no-inf-dups_ellipap_iter.out'

objID_masses = np.loadtxt(fpath+file, dtype = str, usecols = [0], unpack = True)
MH2_inner, MH2_outer, MH2_total, MH2_inner_err, MH2_outer_err, MH2_total_err, f_inner, f_outer = np.loadtxt(fpath+file, dtype = float, usecols = [1,2,3,4,5,6,7,8], unpack = True)

In [6]:
d2 = {'objID': objID_masses, 'MH2_total': MH2_total, 'MH2_total_err': MH2_total_err, 'MH2_inner': MH2_inner, 'MH2_inner_err': MH2_inner_err, 'MH2_outer': MH2_outer, 'MH2_outer_err': MH2_outer_err, 'f_inner': f_inner, 'f_outer': f_outer}
df2 = pd.DataFrame(data=d2)

# join the catalogues
df3 = df.set_index('objID').join(df2.set_index('objID'), on = 'objID')
df3

,objID,MH2_total,MH2_total_err,MH2_inner,MH2_inner_err,MH2_outer,MH2_outer_err,f_inner,f_outer
0,587722982291865751,9.718864,8.402953,8.875613,7.453678,9.650806,8.407058,1.000000,0.073013
1,587722984438235345,9.639653,9.063728,9.004281,8.342456,9.568404,9.056705,0.000000,0.000000
2,587722984440135850,9.457164,8.221691,8.831736,7.306862,9.340156,8.228246,0.916667,0.053456
3,587722984441250023,9.264568,8.698139,9.129479,8.276146,8.835360,8.693503,0.000000,0.000000
4,587724197746311240,10.045982,8.907321,9.708447,7.914810,9.781989,8.902986,1.000000,0.010094
...,...,...,...,...,...,...,...,...,...
268,588848900465885460,9.669512,8.627987,9.071339,7.789696,9.541535,8.649681,0.783333,0.014801
269,588848900465950856,8.468170,7.844903,7.961970,7.067588,8.444257,7.963376,0.616667,0.007777
270,588848900466213053,8.497887,8.241732,5.151758,6.770758,8.531690,8.253193,0.000000,0.000000
271,588848900966449330,9.922120,9.444452,9.470637,8.133410,9.753799,9.439038,1.000000,0.168161


## 3. Add visual classification of SALVAGE images and spectra

In [12]:
fpath = '/arc/projects/salvage/ALMA_reduction/'
file = 'SALVAGE_classification.csv'

objID_classes, morph_class, spec_class = np.loadtxt(fpath+file, dtype = str, unpack = True, delimiter = ',', usecols = [0,1,2], skiprows = 1)

In [14]:
d4 = {'objID': objID_classes, 'visual_class': morph_class, 'spec_class': spec_class}
df4 = pd.DataFrame(data=d4)

# join the catalogues
df5 = df3.join(df4.set_index('objID'), on = 'objID')
df5

,objID,visual_class,spec_class
0,587722982291865751,M,A
1,587722984438235345,N,B
2,587722984440135850,N,A
3,587722984441250023,N,C
4,587724197746311240,M,A
...,...,...,...
289,588848900465950856,N,B
290,588848900466213053,N,C
291,588848900966449330,M,A
292,588848900966514994,N,


## 4. Basic SDSS parameters from local UVic DR7 catalogues

In [16]:
#Query MySQL
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')

x = 'SELECT u.objID, u.total_mass_med, u.fiber_mass_med, u.total_sfr_med, u.fiber_sfr_med, u.z_spec,\
            u.agn_sn, u.new_K03_flag, u.new_K01_flag, u.agn_sn_mw, u.S06_flag_mw, u.K03_flag_mw, u.K01_flag_mw, u.vdisp, u.balmer_sn\
     FROM dr7_uberuber u WHERE u.total_mass_med > 1'

c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
db_data = np.array(db_data, dtype = str).T

table_id = db_data[0]

# inner and outer masses
mass_tot = np.array(db_data[1], dtype = float)
mass_fib = np.array(db_data[2], dtype = float)
sfr_tot = np.array(db_data[3], dtype = float)
sfr_fib = np.array(db_data[4], dtype = float)
mass_out = np.log10(10**mass_tot - 10**mass_fib)
sfr_out = np.log10(10**sfr_tot - 10**sfr_fib)

# spectroscopic redshift
z_sdss = np.array(db_data[5], dtype = float)

# Optical AGN classifications
agn_sn = np.array(db_data[6], dtype = str)
agn_sn[agn_sn=='None'] = -1
agn_sn = np.array(agn_sn, dtype = float)
K03 = np.array(db_data[7], dtype = str)
K01 = np.array(db_data[8], dtype = str)

# Optical AGN classifications (corrected using mw extinction curve)
agn_sn_mw = np.array(db_data[9], dtype = str)
agn_sn_mw[agn_sn_mw=='None'] = -1
agn_sn_mw = np.array(agn_sn_mw, dtype = float)
S06_mw = np.array(db_data[10], dtype = str)
K03_mw = np.array(db_data[11], dtype = str)
K01_mw = np.array(db_data[12], dtype = str)

# velocity dispersion
vdisp = np.array(db_data[13], dtype = float)

# balmer_sn
balmer_sn = np.array(db_data[14], dtype = str)
balmer_sn[balmer_sn=='None'] = -1
balmer_sn = np.array(balmer_sn, dtype = float)

/tmp/ipykernel_20432/4047331550.py:24: RuntimeWarning: divide by zero encountered in log10
  mass_out = np.log10(10**mass_tot - 10**mass_fib)
/tmp/ipykernel_20432/4047331550.py:24: RuntimeWarning: invalid value encountered in log10
  mass_out = np.log10(10**mass_tot - 10**mass_fib)
/tmp/ipykernel_20432/4047331550.py:25: RuntimeWarning: invalid value encountered in log10
  sfr_out = np.log10(10**sfr_tot - 10**sfr_fib)


In [17]:
# create a mw-corrected hybrid, where lines are only mw-corrected if balmer_sn > 3

agn_sn_mw_bsn = agn_sn_mw.copy()
agn_sn_mw_bsn[balmer_sn<3] = agn_sn[balmer_sn<3]

K03_mw_bsn = K03_mw.copy()
K03_mw_bsn[balmer_sn<3] = K03[balmer_sn<3]

K01_mw_bsn = K01_mw.copy()
K01_mw_bsn[balmer_sn<3] = K01[balmer_sn<3]

In [18]:
d6 = {'objID': table_id, 'mstar_inner': mass_fib, 'sfr_inner': sfr_fib, 'mstar_outer': mass_out, 'sfr_outer': sfr_out, 'mstar_outer': mass_out, 'sfr_total': sfr_tot, \
      'K03_lauca': K03, 'K01_lauca': K01, 'agn_sn': agn_sn, 'balmer_sn': balmer_sn, \
      'S06_lauca_mw': S06_mw, 'K03_lauca_mw': K03_mw, 'K01_lauca_mw': K01_mw, 'agn_sn_mw': agn_sn_mw, \
      'K03_lauca_mw_bsn': K03_mw_bsn, 'K01_lauca_mw_bsn': K01_mw_bsn, 'agn_sn_mw_bsn': agn_sn_mw_bsn, \
      'vdisp': vdisp}
df6 = pd.DataFrame(data=d6)

df7 = df5.join(df6.set_index('objID'), on = 'objID')
df7

,objID,mstar_inner,sfr_inner,mstar_outer,sfr_outer,sfr_total,K03_lauca,K01_lauca,agn_sn,balmer_sn,S06_lauca_mw,K03_lauca_mw,K01_lauca_mw,agn_sn_mw,K03_lauca_mw_bsn,K01_lauca_mw_bsn,agn_sn_mw_bsn,vdisp
0,587722953304310243,9.47997,-1.603380,9.992561,-1.164663,-1.029800,AGN,SF,1.70629,1.75380,AGN,AGN,SF,2.67205,AGN,SF,1.70629,122.729
1,587722953304310184,10.34830,-1.535400,10.667324,-0.877563,-0.791250,NULL,NULL,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,135.416
2,587722953304310091,10.26990,-0.228897,10.211691,-0.973782,-0.157039,NULL,NULL,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,177.879
3,587722953304179134,10.59850,-0.027715,11.102010,-0.567031,0.082490,NULL,NULL,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,222.414
4,587722953304179097,9.71313,0.118628,10.280915,0.618732,0.738038,SF,SF,2.89088,9.74048,SF,SF,SF,4.52712,SF,SF,4.52712,-9999.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674630,588848900972413059,9.92884,0.398303,10.345415,0.379927,0.690242,SF,SF,4.10051,10.71010,SF,SF,SF,6.42141,SF,SF,6.42141,-9999.000
674631,588848900972150967,10.71910,-1.317530,11.092810,-0.732465,-0.632102,NULL,NULL,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,149.139
674632,588848900972281999,9.30709,-0.603690,9.632285,0.016902,0.110168,SF,SF,11.49850,14.06020,SF,SF,SF,18.00660,SF,SF,18.00660,-9999.000
674633,588848900972282016,10.74400,-1.280890,11.229687,-0.527753,-0.457143,NULL,NULL,-1.00000,-1.00000,None,None,None,-1.00000,NULL,NULL,-1.00000,-9999.000


## 4. 

## 4. 

## 4. 

## 4. 

## 4. 

## 4. 

## 4. 

## 4. 